In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from IPython.display import Markdown,display
import datetime
import time
import re
from data_helper import *
import os

In [2]:
file_path = 'data/data_asset_choosed/'
file_name = '2371.csv'
df_asset = pd.read_csv(file_path+file_name)
keys = ['MDI_OBD_SPEED','GPS_SPEED','MDI_OBD_RPM','MDI_OBD_ENGINE_LOAD',
        'MDI_OBD_FUEL','ODO_FULL_METER','MDI_DASHBOARD_MILEAGE']

df_asset = fill_blank_data_of_last_value(df_asset,'MDI_OBD_SPEED',145)
df_asset = fill_blank_data_of_last_value(df_asset,'GPS_SPEED',145)
df_asset = fill_blank_data_of_last_value(df_asset,'MDI_OBD_RPM',145)
df_asset = fill_blank_data_of_last_value(df_asset,'MDI_OBD_ENGINE_LOAD',145)
df_asset = fill_blank_data_of_last_value(df_asset,'MDI_OBD_FUEL',145,10)
df_asset = fill_blank_data_of_last_value(df_asset,'ODO_FULL_METER',145,10)
df_asset = fill_blank_data_of_last_value(df_asset,'MDI_DASHBOARD_MILEAGE',145,1)
df_asset.to_csv(file_path+'2371_filled.csv',index=False)

In [16]:
def generate_info(df_asset):
    df_asset["fuel"]=""
    df_asset["distance"]=""
    df_asset["average_speed"]=""
    for i in range(1,len(df_asset)):
        current_data   = df_asset.iloc[i]
        last_data      = df_asset.iloc[i-1]
        current_fuel   = current_data["MDI_OBD_FUEL"]
        current_meter  = current_data["ODO_FULL_METER"]
        current_mil    = current_data["MDI_DASHBOARD_MILEAGE"]
        current_time   = time.mktime( datetime.datetime.strptime(current_data["recorded_at"], "%Y-%m-%dT%H:%M:%SZ").timetuple() )
        last_fuel      = last_data["MDI_OBD_FUEL"]
        last_meter     = last_data["ODO_FULL_METER"]
        last_mil       = last_data["MDI_DASHBOARD_MILEAGE"]
        last_time      = time.mktime( datetime.datetime.strptime(last_data["recorded_at"], "%Y-%m-%dT%H:%M:%SZ").timetuple() )

        if current_time - last_time<=145:
            if current_meter!=' ' and last_meter!=' ':
                distance = int( re.search("\d+",current_meter).group() ) - int( re.search("\d+",last_meter).group() )
                if current_time - last_time!=0:
                    average_speed = distance/1000/(current_time - last_time)*3600
                else:
                    average_speed = df_asset.loc[i-1, "average_speed"]
                df_asset.loc[i, "distance"] = distance
                df_asset.loc[i, "average_speed"] = average_speed
            if current_fuel!=' ' and last_fuel!=' ':
                fuel     = int( re.search("\d+",current_fuel).group() ) - int( re.search("\d+",last_fuel).group() )
                df_asset.loc[i, "fuel"] = fuel
    return df_asset
    
file_path = 'data/data_asset_choosed/'
file_name = '2371_filled.csv'
df_asset = pd.read_csv(file_path+file_name)
df_asset = generate_info(df_asset)
df_asset.to_csv(file_path+'2371_filled_test.csv',index=False)

In [10]:
def generate_speed_info_original_data(df_asset):
    df_asset["fuel"]=""
    df_asset["distance"]=""
    df_asset["average_speed"]=""
    for i in range(1,len(df_asset)):
        current_data   = df_asset.iloc[i]
        current_meter  = current_data["ODO_FULL_METER"]
        current_time   = time.mktime( datetime.datetime.strptime(current_data["recorded_at"], "%Y-%m-%dT%H:%M:%SZ").timetuple() ) 
        key = "ODO_FULL_METER"
        
        if current_meter == " ":
            last_valide_index = None
            last_valide_time  = None
            last_valide_value = None
            next_valide_index = None
            next_valide_time  = None
            next_valide_value = None
            for j in reversed(range(i)):
                    if df_asset.iloc[j][key] != ' ':
                        last_valide_index = j
                        last_valide_time  = time.mktime( datetime.datetime.strptime(df_asset.iloc[j]['recorded_at'],"%Y-%m-%dT%H:%M:%SZ").timetuple() )
                        last_valide_value = current_value  = df_asset.iloc[j][key]
                        break

            for k in range(i+1,len(df_asset)):
                if df_asset.iloc[k][key] != ' ':
                    next_valide_index = k
                    next_valide_time  = time.mktime( datetime.datetime.strptime(df_asset.iloc[k]['recorded_at'],"%Y-%m-%dT%H:%M:%SZ").timetuple() )
                    next_valide_value = current_value  = df_asset.iloc[k][key]
                    break
                    
            if last_valide_value != None and next_valide_value != None :
                interval_time  = next_valide_time  - last_valide_time
                interval_value = int(next_valide_value) - int(last_valide_value)
                
                if interval_time<=145:
                    if interval_time == 0:
                        df_asset.loc[i,"average_speed"] = df_asset.loc[i-1,"average_speed"]
                    else:
                        df_asset.loc[i,"average_speed"] = str(interval_value/interval_time*3.6)
                
                    
        else:
            last_valide_index = None
            last_valide_time  = None
            last_valide_value = None
            for j in reversed(range(i)):
                    if df_asset.iloc[j][key] != ' ':
                        last_valide_index = j
                        last_valide_time  = time.mktime( datetime.datetime.strptime(df_asset.iloc[j]['recorded_at'],"%Y-%m-%dT%H:%M:%SZ").timetuple() )
                        last_valide_value = current_value  = df_asset.iloc[j][key]
                        break
            
            if last_valide_value != None :
                interval_time  = current_time - last_valide_time
                interval_value = int(current_meter) - int(last_valide_value)
                
                if interval_time<=145:
                    if interval_time == 0:
                        df_asset.loc[i,"average_speed"] = df_asset.loc[i-1,"average_speed"]
                    else:
                        df_asset.loc[i,"average_speed"] = str(interval_value/interval_time*3.6)
    return df_asset 



df_asset = generate_speed_info_original_data(df_asset)
df_asset.to_csv(file_path+'2371_test.csv',index=False)          